In [1]:
import cv2 
import cv2.aruco as aruco

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
!ls /dev | grep video

video2
video3


In [2]:
cam_id = 2
capture = cv2.VideoCapture(cam_id)

In [ ]:
# capture.release()

In [19]:
res = (1280, 720)
res_x, res_y = res
capture.set(3, res_x)
capture.set(4, res_y)
capture.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))

True

In [ ]:
from pathlib import Path

def get_path(path):
    path = Path(path)
    if not path.exists():
        return path
    raise ValueError("File Exists!")

In [21]:
import numpy as np
import time

TILE_SIZE = .2391 / 8
MARKER_SIZE = TILE_SIZE * 23 / 30


class Camera:

    def __init__(self, tile_size=TILE_SIZE, marker_size=MARKER_SIZE,
                 board_size=(5, 8),):
        self.captured_images = []

        self.dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
        self.parameters = aruco.DetectorParameters()
        self.charuco_params = aruco.CharucoParameters()

        self.tile_size = tile_size
        self.board = aruco.CharucoBoard(board_size, self.tile_size, marker_size,
                                        self.dictionary)
        self.board.setLegacyPattern(True)
        self.detector = aruco.CharucoDetector(
            self.board, self.charuco_params, self.parameters)

        self.mtx, self.dist = None, None
        self.captured_images = []
        self.last_time = 0

    def add_frame(self, image) -> np.ndarray:
        """
        returns RGBA array of size (x, y, 4)
        """
        image0 = image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        font = cv2.FONT_HERSHEY_SIMPLEX

        cur_corners, cur_ids, _ = self.detector.detectMarkers(image)

        if time.time() - self.last_time < 0.5:
            cv2.circle(image0, (1150, 700), 10, (255, 255, 0),
                       int(np.sin(time.time() - self.last_time) * 40) + 1)
        elif len(cur_corners) >= 19:
            cv2.circle(image0, (1200, 700), 10, (255, 255, 0), 2)
            self.captured_images.append(image0.copy())
            self.last_time = time.time()

        cv2.putText(image0, f"{len(cur_corners):02d}",
                    (64, 64), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
        return image0

    def load_coeffs(self):
        # self.mtx = np.array([[1.11297931e+03, 0.00000000e+00, 6.40000000e+02],
        #                      [0.00000000e+00, 1.11297931e+03, 3.60000000e+02],
        #                      [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
        # self.dist = np.zeros((1, 5))
        self.mtx = np.array([[1.00604718e+03, 0.00000000e+00, 6.35524500e+02],
                             [0.00000000e+00, 1.00242528e+03, 3.65893138e+02],
                             [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
        self.dist = np.array(
            [[0.12490427, -0.38032263, -0.00073625, -0.00364679,  0.34870377]])

    def calibrate(self):
        matched_points = []
        for image in self.captured_images:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cur_corners, cur_ids, *_ = self.detector.detectBoard(image)
            if cur_ids is not None:
                matched_points.append(list(
                    map(np.squeeze, self.board.matchImagePoints(
                        cur_corners, cur_ids.flatten()))
                ))
        obj_points, img_points = zip(*matched_points)
        print(obj_points[1].shape, len(img_points),
              obj_points[0].shape, img_points[0].shape)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
            obj_points, img_points, image.shape,
            None, None
        )
        self.mtx = mtx
        self.dist = dist
        print()
        print(self.mtx, self.dist)
        print()

    def process_charuko(self, frame):
        assert self.mtx is not None and self.dist is not None

        frame = frame.copy()

        font = cv2.FONT_HERSHEY_SIMPLEX
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        *_, corners, ids = self.detector.detectBoard(gray)

        if not len(corners):
            # cv2.putText(frame, "No Ids", (0,64), font, 1, (0,255,0),2,cv2.LINE_AA)
            return frame, None, None

        _, corners_checker, ids_checker = aruco.interpolateCornersCharuco(
            corners, ids,
            gray, self.board
        )

        _, rvec_check, tvec_check = aruco.estimatePoseCharucoBoard(
            corners_checker, ids_checker,
            self.board, self.mtx, self.dist,
            None, None
        )
        try:
            cv2.drawFrameAxes(frame, self.mtx, self.dist,
                              rvec_check, tvec_check, 0.1)
        except:
            pass
        # print(rvec_check, tvec_check)
        aruco.drawDetectedMarkers(frame, corners)
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(
            corners, self.tile_size / 2, self.mtx, self.dist)
        # print()
        # print(rvecs[0], tvecs[0])

        for rvec, tvec in zip(rvecs, tvecs):
            cv2.drawFrameAxes(frame, self.mtx, self.dist, rvec, tvec, 0.01)

        return frame, rvec_check, tvec_check

In [4]:

# with open("./data/images_1.npy", "rb") as f:
#     calibr.captured_images.extend(list(np.load(f)))
# with open("./data/images_2.npy", "rb") as f:
#     calibr.captured_images.extend(list(np.load(f)))

In [5]:
from websocket import create_connection
import json
def get_pos():
    req = {"command": "get_xyz", "args":[], "kwargs":{}}
    data = json.dumps(req)
    ws = create_connection("ws://localhost:9001")
    ws.send(data)
    res = json.loads(ws.recv())
    ws.close()
    return res

In [6]:
np.set_printoptions(precision=2)

In [17]:
end_effector_poses = []
cam_to_checkers = []


def append_ef(xyz):
    x, y, z = np.array(xyz) / 1000
    alpha = np.arctan2(y, x)
    matrix = np.array([[np.cos(alpha), -np.sin(alpha), 0, x],
                       [np.sin(alpha), np.cos(alpha), 0, y],
                       [0, 0, 1, z],
                       [0, 0, 0, 1]])
    end_effector_poses.append(matrix)

def get_twist(R, p):
    return np.block([[R, p.reshape(-1, 1)], [0] * len(p) + [1]])
    
def append_cam2checker(rvec, tvec):
    cam_to_checkers.append(get_twist(cv2.Rodrigues(rvec)[0], tvec))

In [15]:
import sys
sys.path.insert(0, "..")
from eye_hand_calibration import get_board2base_transform

In [34]:
capture.read()

(False, None)

In [22]:
def _stream():
    displaying_aruco = False
    while True:
        _, cam_img = capture.read()
        if not displaying_aruco:
            cv2.imshow('w', cam_img)
        else:
            cv2.imshow('w', calibr.process_charuko(cam_img.copy())[0])
        
        if (key := cv2.waitKey(1)) == ord("q"):
            break
        elif key == ord("n"):
            displaying_aruco = not displaying_aruco
        elif key == ord("f"):
            _, rvec, tvec = calibr.process_charuko(cam_img.copy())
            xyz = get_pos()["result"]
            append_cam2checker(rvec, tvec)
            append_ef(xyz)
def stream():
    try:
        _stream()
    finally:
        cv2.destroyAllWindows()

In [5]:
capture.release()

In [23]:
calibr = Camera(board_size=(5, 8))
calibr.load_coeffs()

In [24]:
stream()

error: OpenCV(4.8.0) /home/conda/feedstock_root/build_artifacts/libopencv_1694892514749/work/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


In [19]:
from IPython.display import clear_output

cam_id = "http://192.168.43.1:8021/video"
capture = cv2.VideoCapture(cam_id)
calibr = Camera(board_size=(5, 8))
calibr.load_coeffs()

stream()
clear_output()

In [25]:
with get_path("0.npy").open("wb") as f:
    np.save(f, end_effector_poses)

In [26]:
with get_path("checker.npy").open("wb") as f:
    np.save(f, cam_to_checkers)


In [30]:
for m in cam_to_checkers:
    np.linalg.inv(m)

In [28]:
for m in end_effector_poses:
    np.linalg.inv(m)


In [34]:
get_pos()

{'result': [-18.81871223449707, 227.0407257080078, -20.088851928710938]}

In [31]:
tr_pred = get_board2base_transform(end_effector_poses, cam_to_checkers)

LinAlgError: Singular matrix

In [24]:
from pathlib import Path

def get_path(path):
    path = Path(path)
    if not path.exists():
        return path
    raise ValueError("File Exists!")

In [ ]:
capture.read()

In [ ]:
stream()

In [ ]:
len(calibr.captured_images)

In [ ]:
calibr.calibrate()

In [ ]:
image = cv2.cvtColor(cam_img, cv2.COLOR_BGR2GRAY)

In [ ]:
cur_corners, cur_ids, _ = detector.detectMarkers(image)

In [ ]:
len(cur_corners)

In [ ]:
_

In [ ]:
import matplotlib.pyplot as plt
_, cam_img = capture.read()
plt.imshow(cam_img)

In [ ]:
plt.imshow(calibr.process_charuko(cam_img)[0])

In [ ]:
def disl_charuco():
    displaying = np.array([])
    try:
        while True:
            _, cam_img = capture.read()
            
            if not displaying.size:
                cv2.imshow('w', cam_img)
            else:
                cv2.imshow('w', displaying)
            if (key := cv2.waitKey(1)) == ord("q"):
                break
            elif key == ord("t"):
                calibr.get_frame()
            elif key == ord("f"):
                displaying = calibr.process_charuko(cam_img)
    finally:
        cv2.destroyAllWindows()

In [ ]:
len(calibr.captured_images)

In [ ]:
calibr.stream()

In [ ]:
chr(123)